This is a data set that I picked out from kaggle. This can be found at: https://www.kaggle.com/rtatman/did-it-rain-in-seattle-19482017

Pick a dataset. It could be one you've worked with before or it could be a new one. Then build the best decision tree you can.

Now try to match that with the simplest random forest you can. For our purposes measure simplicity with runtime. Compare that to the runtime of the decision tree. This is imperfect but just go with it.

Hopefully out of this you'll see the power of random forests, but also their potential costs. Remember, in the real world you won't necessarily be dealing with thousands of rows. It could be millions, billions, or even more.

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import ensemble
from sklearn import tree
import pydotplus
import graphviz
import time
%matplotlib inline

In [3]:
df = pd.read_csv('seattleWeather_1948-2017.csv')

In [7]:
print(df.shape)
df.head()

(25551, 5)


,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True


In [11]:
df.dtypes

DATE     object
PRCP    float64
TMAX      int64
TMIN      int64
RAIN     object
dtype: object

In [12]:
# Change Rain to binary in order to build model.
df['RAIN'] = df['RAIN'].map(lambda i: 1 if i == True else 0)

In [16]:
# Had missing values before so going to drop some valeus 
df = df.dropna()

In [20]:
df.head(80)

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,1
1,1948-01-02,0.59,45,36,1
2,1948-01-03,0.42,45,35,1
3,1948-01-04,0.31,45,34,1
4,1948-01-05,0.17,45,32,1
5,1948-01-06,0.44,48,39,1
6,1948-01-07,0.41,50,40,1
7,1948-01-08,0.04,48,35,1
8,1948-01-09,0.12,50,31,1
9,1948-01-10,0.74,43,34,1


In [23]:
# decision tree
# Initialize and train our tree.
X = df.drop(['RAIN'], axis = 1)
X = pd.get_dummies(X)
Y = df['RAIN']

dt_start_time = time.time()
decision_tree = tree.DecisionTreeClassifier(
    criterion='entropy',
    max_features=2,
    max_depth=5,
    random_state = 1337
)
decision_tree.fit(X, Y)
display(cross_val_score(decision_tree, X, Y, cv = 5))
print ("Decision Tree runtime: {}".format(time.time() - dt_start_time))

array([0.57338552, 0.57338552, 0.57338552, 0.57330202, 0.57330202])

Decision Tree runtime: 38.749423027038574


In [25]:
# random forest
rf_start_time = time.time()
rfc = ensemble.RandomForestClassifier()
display(cross_val_score(rfc, X, Y, cv=5))
print ("Random Forest runtime: {}".format(time.time() - rf_start_time))

array([1., 1., 1., 1., 1.])

Random Forest runtime: 74.83951497077942


My random forest model is extremely accurate which makes me wonder if I overfit, however, it took nearly double the amount of time than the simple decision tree. 